# Laboratorio 4 - Inteligencia Artificial

## Autores:

- Nelson García
- Andy Fuentes
- Davis Roldan
- Ricardo Chuy
- Diederich Solís

## Task 1 - Preguntas teóricas

## Task 2 - Ejercicios Prácticos

### Task 2.1 - K-Means y Segmentación de Clientes de Banco

#### Task 2.1.1 - K-Means

Se importan las librerías a utilizar.

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

Se lee el dataset.

In [12]:
dataset = pd.read_csv('documents/bank_transactions.csv')

Visualizar los datos.

In [13]:
dataset.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [14]:
dataset.describe()

,CustAccountBalance,TransactionTime,TransactionAmount (INR)
count,1.046198e+06,1.048567e+06,1.048567e+06
mean,1.154035e+05,1.570875e+05,1.574335e+03
std,8.464854e+05,5.126185e+04,6.574743e+03
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.721760e+03,1.240300e+05,1.610000e+02
50%,1.679218e+04,1.642260e+05,4.590300e+02
75%,5.765736e+04,2.000100e+05,1.200000e+03
max,1.150355e+08,2.359590e+05,1.560035e+06


In [15]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1045170 non-null  object 
 3   CustGender               1047467 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB


##### Elección de variables

Saldo de la Cuenta del Cliente (CustAccountBalance)

El saldo de la cuenta es un indicador clave del poder adquisitivo y el comportamiento financiero del cliente. Puede ayudar a distinguir entre clientes de alto, medio y bajo poder adquisitivo, lo que es crucial para segmentación en productos bancarios.

Edad del Cliente (CustomerDOB → Convertido a Edad)

La edad es un fuerte diferenciador en el comportamiento financiero y preferencias bancarias. Jóvenes pueden tener un comportamiento transaccional diferente al de clientes mayores, lo que permite agruparlos en segmentos con necesidades bancarias similares.

Convertir CustomerDOB a edad de cada cliente

In [ ]:
# Función para corregir el año según la lógica dada
def fix_year(dob):
    try:
        day, month, year = map(int, dob.split('/'))
        if year <= 7:  # Años 00-07 son de los 2000s
            year += 2000
        else:  # Años 08-99 son de los 1900s
            year += 1900
        return datetime(year, month, day)
    except:
        return None  # Para valores mal formateados o nulos


In [17]:
# Aplicar la corrección en la columna CustomerDOB
dataset['CustomerDOB'] = dataset['CustomerDOB'].astype(str).apply(fix_year)

# Convertir la columna corregida a datetime
dataset['CustomerDOB'] = pd.to_datetime(dataset['CustomerDOB'], errors='coerce')

# Calcular la edad
today = datetime.today()
dataset['Age'] = dataset['CustomerDOB'].apply(lambda dob: today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day)) if pd.notnull(dob) else None)

# Reemplazar valores NaN en Age con la mediana de la edad calculada
median_age = dataset['Age'].median()
dataset['Age'].fillna(median_age, inplace=True)

# Verificar los primeros valores
print(dataset[['CustomerDOB', 'Age']].head())

  CustomerDOB   Age
0  1994-01-10  31.0
1  1957-04-04  67.0
2  1996-11-26  28.0
3  1973-09-14  51.0
4  1988-03-24  36.0


C:\Users\nel20\AppData\Local\Temp\ipykernel_112\985899336.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset['Age'].fillna(median_age, inplace=True)


In [18]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 10 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   TransactionID            1048567 non-null  object        
 1   CustomerID               1048567 non-null  object        
 2   CustomerDOB              987831 non-null   datetime64[ns]
 3   CustGender               1047467 non-null  object        
 4   CustLocation             1048416 non-null  object        
 5   CustAccountBalance       1046198 non-null  float64       
 6   TransactionDate          1048567 non-null  object        
 7   TransactionTime          1048567 non-null  int64         
 8   TransactionAmount (INR)  1048567 non-null  float64       
 9   Age                      1048567 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 80.0+ MB


In [19]:
dataset.describe()

,CustomerDOB,CustAccountBalance,TransactionTime,TransactionAmount (INR),Age
count,987831,1.046198e+06,1.048567e+06,1.048567e+06,1.048567e+06
mean,1985-06-13 10:37:22.389234880,1.154035e+05,1.570875e+05,1.574335e+03,3.905388e+01
min,1908-04-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,1.700000e+01
25%,1982-02-06 00:00:00,4.721760e+03,1.240300e+05,1.610000e+02,3.400000e+01
50%,1987-09-12 00:00:00,1.679218e+04,1.642260e+05,4.590300e+02,3.700000e+01
75%,1991-05-11 00:00:00,5.765736e+04,2.000100e+05,1.200000e+03,4.200000e+01
max,2007-12-12 00:00:00,1.150355e+08,2.359590e+05,1.560035e+06,1.160000e+02
std,NaN,8.464854e+05,5.126185e+04,6.574743e+03,8.566647e+00


Ahora que se tiene la edad se puede comenzar el proceso de clustering.

In [20]:
X = dataset.filter(['CustAccountBalance', 'Age'])

In [23]:
import scipy.spatial.distance as metric

def euclidean_dist(A, B):
    return metric.euclidean(A, B)

def plot(data,k,index,centroids,orig_centroids):
    input = []
    for i in range(len(index)):
        for j in index[i]:
            input.append(int(j))
            
    colors = 10*["g","r","c","b","k"]
    j=0
    for i in input:
        plt.scatter(data[0][j,0], data[0][j,1], marker="x", color=colors[i], s=150, linewidths=5)
        j+=1
    ## New centroids
    for centroid in range(len(centroids)):
        plt.scatter(centroids[centroid][0],centroids[centroid][1],marker="o", color="k", s=150, linewidths=5)
    # Original Clusters
    for centroid in range(len(orig_centroids)):
        plt.scatter(orig_centroids[centroid][0],orig_centroids[centroid][1],marker="D", color="DarkBlue", s=150, linewidths=5)

Función principal de k-means

In [24]:
class K_Means:
    def __init__(self,k,data,centeriod_init=None):
        self.k = k
        self.data = data  
        self.centeriod_init = centeriod_init
        
    def initialise_centroids(self,centeriod_init,k,data):
        ## 3 ways to initialize centroides
        if(self.centeriod_init == 'random'): 
            initial_centroids = np.random.permutation(data.shape[0])[:self.k]
            self.centroids = data[initial_centroids]
        elif(self.centeriod_init == 'firstk'):
            self.centroids = data[:k]
        else:
            for i in range(self.k):
                self.centroids.append(i%self.k)
        return self.centroids    
 
    def fit(self,data):
        m = np.shape(data)[0]
        cluster_assignments = np.mat(np.zeros((m,2)))
        
        cents = self.initialise_centroids(self.centeriod_init,self.k,data)
        
        # Preserve original centroids
        cents_orig = cents.copy()
        changed = True
        num_iter = 0
        
        while changed and num_iter<100:
            changed = False 
            # for each row in the dataset
            for i in range(m):
                # Track minimum distance and vector index of associated cluster
                min_dist = np.inf
                min_index = -1 
                #calculate distance 
                for j in range(self.k):
                    dist_ji = euclidean_dist(cents[j,:],data[i,:])
                    if(dist_ji < min_dist):
                        min_dist = dist_ji
                        min_index = j 
                    # Check if cluster assignment of instance has changed
                    if cluster_assignments[i, 0] != min_index: 
                        changed = True

                # Assign instance to appropriate cluster
                cluster_assignments[i, :] = min_index, min_dist**2

            # Update centroid location
            for cent in range(self.k):
                points = data[np.nonzero(cluster_assignments[:,0].A==cent)[0]]
                cents[cent,:] = np.mean(points, axis=0)
    
            # Count iterations
            num_iter += 1
            #print(num_iter)

         # Return important stuff when done
        return cents, cluster_assignments, num_iter, cents_orig

Implementación de k-means:

In [25]:
kmeans = K_Means(k=3,data = X,centeriod_init='random')
centroids, cluster_assignments, iters, orig_centroids = kmeans.fit(X)
index = cluster_assignments[:,0] ## This has the cluster assignment 0,1,.... 
distance = cluster_assignments[:,1]  ## This has the distance from their respective centroides for evaluation purposes 
k=3
plot(X,k,index,centroids,orig_centroids)

KeyError: "None of [Index([132652, 430252, 401134], dtype='int32')] are in the [columns]"

### Task 2.2 - Mixture Modelos y Segmentación de Clientes de Banco